In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import more library and load the data

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

In [ ]:
# Thanks for ROB MULLA's dataset and code so Let's load the data with parquet files
# details can be found at https://www.kaggle.com/robikscube/fast-data-loading-and-low-mem-with-parquet-files
train = pd.read_parquet('../input/ubiquant-parquet/train_low_mem.parquet')
train.head(10)

# Missing values check

In [ ]:
train.isnull().any().sum()

Great, there isn't any missing values here, and next we are going to dig some deeper information there. Here follows the description of the columns in the train.csv

# Analysis and feature engineering

train.csv

* row_id - A unique identifier for the row.
* time_id - The ID code for the time the data was gathered. The time IDs are in order, but the real time between the time IDs is not constant and will likely be shorter for the final private test set than in the training set.
* investment_id - The ID code for an investment. Not all investment have data in all time IDs.
* target - The target.
[f_0:f_299] - Anonymized features generated from market data.

In [ ]:
# Let's first take a look at the first 3 columns
print('The number of unique value in the first column here is {}.'.format(train.row_id.nunique()))
print('The number of unique value in the time_id column here is {}.'.format(train.time_id.nunique()))
print('The number of unique value in the investment_id column here is {}.'.format(train.investment_id.nunique()))

It can be easily found that there are significant amount of repeated value in the time_id and investment_id column, so let's dig deeper in that

In [ ]:
plt.figure(figsize = (20,10))
time_id = pd.DataFrame(train['time_id'].value_counts())
time_id = time_id.rename(columns ={'time_id': 'counts'})
sns.barplot(data = time_id, y = 'counts', x = time_id.index)
time_id

**Notice that, here, each count of time_id means that there existed a investment instruction
It shows that for small time id, it has a constant number of investment instruction, but it will reduce at some point, and then it keep going up after that time point.**

In [ ]:
# max investment_id for specific time_id
train.groupby("time_id").investment_id.max().value_counts()

all the value here are large so we omit this part

In [ ]:
plt.figure(figsize = (20,10))
investment_id = pd.DataFrame(train['investment_id'].value_counts())
investment_id = investment_id.rename(columns ={'investment_id': 'counts'})
sns.barplot(data = investment_id, x = investment_id.index, y = investment_id.counts)

In [ ]:
# max time_id for specific investment_id
train.groupby("investment_id").time_id.max().value_counts()

In [ ]:
print(train.groupby('investment_id').time_id.max().value_counts().index.max())
print(train.groupby('investment_id').time_id.max().value_counts().index.min())

Notice that 62 << 1219, we know that for some investments, it ended significantly earlier than others.

In [ ]:
from sklearn.feature_selection import mutual_info_regression
def make_mi_score_real(X, y):
    mi_scores = mutual_info_regression(X,y)
    mi_scores = pd.Series(mi_scores, name = 'MI scores', index = X.columns)
    mi_scores = mi_scores.sort_values(ascending = False)
    return mi_scores
column_for_regression  = []

In [ ]:
for i in range(20):
    traincopy = train.copy()
    y = traincopy.pop('target')[20000*i:20000*(i+1)]
    y.dtype
    y = y.astype(np.float64)
    a = traincopy.iloc[20000*i:20000*(i+1)]
    mi_scores_real = make_mi_score_real(a,y)
    mi_scores_real.round(decimals=4)
    for j in range(6):
        column_for_regression.append(mi_scores_real.index[j])

In [ ]:
print('The length of the list is {}'.format(len(column_for_regression)))
print('And then we convert it to set.')
column_for_regression.append('target')
column_for_regression.append('time_id')
column_for_regression.append('investment_id')
b = list(set(column_for_regression))
print('The lenth of the set is {}'.format(len(b)))

In [ ]:
new_train = train.copy()[b]
target = new_train.pop('target')

In [ ]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
X_train, X_test, y_train, y_test = train_test_split(new_train, target, test_size=0.2, random_state=0)
model = XGBRegressor(learning_rate=0.01,n_estimators=3500,
                                     max_depth=3, min_child_weight=0,
                                     gamma=0, subsample=0.7,
                                     colsample_bytree=0.7,
                                     objective='reg:linear', nthread=-1,
                                     scale_pos_weight=1, seed=27,
                                     reg_alpha=0.0001)
model.fit(X_train, y_train, early_stopping_rounds=10, eval_set=[(X_test, y_test)])

In [ ]:
print('all set')

To be continued

In [ ]:
with open('model_lr.pickle', 'wb') as f:
    pickle.dump(model, f)
with open('feature_names.pickle', 'wb') as f:
    pickle.dump(b, f)